In [1]:
import numpy as np

In [2]:
img_test = np.array([
    [1.0, 2.0, 1.0, 1.0],
    [2.0, 0.0, 0.0, 3.0],
    [1.0, 1.0, 2.0, 0.0],
    [2.0, 2.0, 0.5, 1.0],
    [2.0, 1.0, 1.0, 0.0]
])

grad_test = np.array([
    [1.0, 1.0 , 0.5, 0.0],
    [2.0, 1.0 , 1.5, 3.0],
    [0.0, 0.5 , 1.5, 2.0],
    [0.0, 0.75, 1.0, 0.5],
    [1.0, 0.5 , 0.5, 1.0]
])

seam_test = np.array([2, 1, 0, 0, 1])

img_red_test = np.array([
    [1.0, 2.0, 1.0],
    [2.0, 0.0, 3.0],
    [1.0, 2.0, 0.0],
    [2.0, 0.5, 1.0],
    [2.0, 1.0, 0.0]
])

In [3]:
def get_grad(img):
    n, m = img.shape

    grad = np.zeros((n, m))
    grad[:, 0] = np.abs(img[:, 0]-img[:, 1])
    grad[:, 1:-1] = (np.abs(img[:, 1:-1]-img[:, 0:-2]) + np.abs(img[:, 1:-1]-img[:, 2:]))/2
    grad[:, -1] = np.abs(img[:, -1]-img[:, -2])

    return grad

def test_get_grad(img, grad_true):
    grad = get_grad(img)
    assert np.array_equal(grad, grad_true)

In [4]:
grad = get_grad(img_test)
print(grad)

test_get_grad(img_test, grad_test)

[[1.   1.   0.5  0.  ]
 [2.   1.   1.5  3.  ]
 [0.   0.5  1.5  2.  ]
 [0.   0.75 1.   0.5 ]
 [1.   0.5  0.5  1.  ]]


In [5]:
def get_seam(grad):
    n, m = grad.shape

    c = np.zeros((n, m))
    c[0] = grad[0]

    c_left, c_top, c_right = np.zeros((n, m)), np.zeros((n, m)), np.zeros((n, m))
    c_min = np.zeros((n, m))
    whence = np.zeros((n, m))

    c_left[0], c_top[0], c_right[0] = np.inf, np.inf, np.inf
    c_min[0], whence[0] = np.inf, np.inf

    for i in range(1, n):
        c_left[i, 0] = np.inf
        c_left[i, 1:] = c[i-1, :-1]

        c_top[i, :] = c[i-1, :]

        c_right[i, :-1] = c[i-1, 1:]
        c_right[i, -1] = np.inf

        c_min[i] = np.min((c_left[i], c_top[i], c_right[i]), axis=0)
        c[i] = c_min[i]+grad[i]

        whence[i, c_min[i] == c_left[i]] = -1
        whence[i, c_min[i] == c_top[i]] = 0
        whence[i, c_min[i] == c_right[i]] = 1

    seam = np.zeros(n, dtype=int)
    js = np.argmin(c[-1, :])

    for i in reversed(range(n)):
        seam[i] = js
        js += whence[i, int (js)]

    return seam

def test_get_seam(grad, seam_true):
    seam = get_seam(grad)
    assert np.array_equal(seam, seam_true)

In [6]:
seam = get_seam(grad_test)
print(seam)

test_get_seam(grad_test, seam_test)

[2 1 0 0 1]


In [7]:
def get_carve(img, seam):
    n, m = img.shape

    img_new = np.zeros((n, m-1))
    for i in range(n):
        js = seam[i]
        img_new[i, :js] = img[i, :js]
        img_new[i, js:] = img[i, js+1:]

    return img_new

def test_get_carve(img, seam, carve_true):
    img_new = get_carve(img, seam)
    assert np.array_equal(img_new, carve_true)

In [8]:
carve = get_carve(img_test, seam)
print(carve)

test_get_carve(img_test, seam, img_red_test)

[[1.  2.  1. ]
 [2.  0.  3. ]
 [1.  2.  0. ]
 [2.  0.5 1. ]
 [2.  1.  0. ]]


In [9]:
def get_seam_carve(img):
    grad = get_grad(img)
    seam = get_seam(grad)
    carve = get_carve(img, seam)

    return carve

def test_seam_carve(img, img_red_true):
    img_new = get_seam_carve(img)
    assert np.array_equal(img_new, img_red_true)

In [10]:
carve = get_seam_carve(img_test)
print(carve)

test_seam_carve(img_test, img_red_test)

[[1.  2.  1. ]
 [2.  0.  3. ]
 [1.  2.  0. ]
 [2.  0.5 1. ]
 [2.  1.  0. ]]
